In [1]:
from jupyter_dash import JupyterDash
from dash import dcc, html
import dash_bootstrap_components as dbc
from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate
from dash_bootstrap_templates import load_figure_template

import plotly.express as px
import pandas as pd
import numpy as np

dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.SLATE, dbc_css])

load_figure_template("SLATE")

education = (pd
             .read_csv("../Data/states_all.csv")
             .iloc[:, 1:]
             .rename({
                 "AVG_MATH_4_SCORE": "4th Gr. Math",
                 "AVG_MATH_8_SCORE": "8th Gr. Math",
                 "AVG_READING_4_SCORE": "4th Gr. Reading",
                 "AVG_READING_8_SCORE": "8th Gr. Reading"
             }, axis=1)
             .assign(expenditure_per_student=lambda x: x["TOTAL_EXPENDITURE"]/x["GRADES_ALL_G"])
)


app.layout = html.Div([
    dbc.Row(html.H1("Education Performance and Expenditure in the US", style={"text-align": "center"})),
    dbc.Row([
       dbc.Col([
           dbc.Card([
           dcc.Markdown("Select an X Column"),
           dcc.RadioItems(
               id="score-radio",
               options=["4th Gr. Math", "8th Gr. Math", "4th Gr. Reading", "8th Gr. Reading"], 
               value="8th Gr. Math"
           ),
           html.Hr(),
           dcc.Markdown("Select a Y Column"),
           dcc.RadioItems(
                id="score-radio2",
                options=["4th Gr. Math", "8th Gr. Math", "4th Gr. Reading", "8th Gr. Reading"],
                value="8th Gr. Reading"
           )])
       ], width=2),
        dbc.Col(
            dcc.Graph(id="cross-filter-scatter", hoverData={'points': [{'customdata': ['CALIFORNIA']}]})
        ),
        dbc.Col(dcc.Graph(id="x-line"))
    ])
])
@app.callback(
    Output("cross-filter-scatter", "figure"),
    Input("score-radio", "value"),
    Input("score-radio2", "value"))
def score_scatter(x, y):
    fig = px.scatter(
        education.query("YEAR == 2013"),
        x=x,
        y=y,
        hover_name="STATE",
        custom_data=["STATE"]
    )
        
    return fig

    
@app.callback(
    Output("x-line", "figure"),
    Input("cross-filter-scatter", "hoverData"))
def update_line(hoverData):
    state_name = hoverData["points"][0]["customdata"][0]
    df=education.query("STATE == @state_name")
    
    fig = px.line(
        df,
        x="YEAR",
        y="expenditure_per_student",
        title=f"Expenditure Per Student in {state_name.title()}"
    ).update_xaxes(showgrid=False)
    
    return fig
    

if __name__ == "__main__":
    app.run_server(debug=True, port=8256)

/Users/pocholo/Desktop/projects/Python_Data_Visualization_Plotly_and_Dash/venv/lib/python3.11/site-packages/dash/dash.py:538: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



Dash app running on http://127.0.0.1:8256/
